In [28]:
# !export PYTHONPATH=/Users/piyush/projects/ViDA-SSL/
import sys
sys.path.append("/Users/piyush/projects/ViDA-SSL/")

In [29]:
from os.path import join
import pandas as pd
from tqdm import tqdm
import numpy as np
import re

import torch
from fast_pytorch_kmeans import KMeans
from transformers import AutoTokenizer, AutoModel

from misc.local_utils import get_phrase_embedding
from utils.viz import bokeh_2d_scatter, bokeh_2d_scatter_new

In [30]:
DATA_DIR = "/Users/piyush/datasets/UCF-101/"
labels_file = join(DATA_DIR, "annotations/action-clf/Class Index.txt")

In [31]:
name = 'CamelCaseName'
name = re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()
print(name)  # camel_case_name

camel_case_name


In [32]:
def camel_to_snake(string):
    """
    Thanks: https://stackoverflow.com/questions/1175208/elegant-python-function-to-convert-camelcase-to-snake-case
    """
    return re.sub(r'(?<!^)(?=[A-Z])', '_', string).lower()

In [19]:
with open(labels_file, "rb") as f:
    lines = f.read()
    lines = lines.decode("utf-8")
    lines = lines.split("\r\n")

class_label_dict = {x.split(" ")[0]: camel_to_snake(x.split(" ")[-1]).replace("_", " ") for x in lines[:-1]}

In [21]:
layers = [-4, -3, -2, -1]

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModel.from_pretrained("bert-base-cased", output_hidden_states=True)

/Users/piyush/miniforge3/envs/video/lib/python3.9/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a Be

In [22]:
eg_embedding = get_phrase_embedding(model, tokenizer, "hello I am a boy")

In [23]:
embeddings = dict()
embeddings_tensor = []
class_ids = []

for k, v in tqdm(class_label_dict.items()):
    phrase_embedding = get_phrase_embedding(model, tokenizer, v)
    embeddings[k] = phrase_embedding
    embeddings_tensor.append(phrase_embedding.unsqueeze(0))
    class_ids.append(k)

embeddings_tensor = torch.cat(embeddings_tensor, dim=0)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible


  1%|█▍                                                                                                                                           | 1/100 [00:00<00:09,  9.93it/s]

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 23.33it/s]


In [24]:
embeddings_tensor.shape

torch.Size([100, 768])

In [25]:
kmeans = KMeans(n_clusters=10, mode='euclidean', verbose=1)
labels = kmeans.fit_predict(embeddings_tensor)

(U, S, V) = torch.pca_lowrank(embeddings_tensor)

K = 2
Z = torch.matmul(embeddings_tensor, V[:, :K])

used 6 iterations (0.0363s) to cluster 100 items into 10 clusters


In [26]:
df = pd.DataFrame(None)

df["x"] = Z[:, 0].numpy()
df["y"] = Z[:, 1].numpy()
df["cluster_label"] = labels.numpy()
df["class_id"] = np.array(class_ids)
df["class_desc"] = df["class_id"].apply(lambda k: class_label_dict[k])

In [27]:
bokeh_2d_scatter_new(
    df=df, x="x", y="y", hue="cluster_label", label="class_desc", use_nb=True,
    title="BERT-based embeddings for FineGym-99 action classes."
)

Loading BokehJS ...